<a href="https://colab.research.google.com/github/ushifali/final-23/blob/master/Collaborative_Filtering_Hotels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
import pandas as pd

# Load user preferences data
user_preferences_df  = pd.read_csv('/content/drive/MyDrive/Filtering_Data/user_preferences.csv', encoding='utf-8')

# Load restaurant data
hotels_df = pd.read_csv('/content/drive/MyDrive/Filtering_Data/content_filtered_data_hotel.csv', encoding='utf-8')

In [18]:
hotels_df.head(5)

,Unnamed: 0,address,area,city,hotel_description,hotel_facilities,hotel_star_rating,image_urls,locality,latitude,...,pageurl,property_name,property_type,province,room_count,room_facilities,room_type,state,rating,Distance
0,9462,"39/5 - Doddanekkundi, Mahadevpura, Marathahall...","Marathahalli,Outer Ring Road,Accenture, RMZ Ec...",Bangalore,|Magic inn is situated at the primal part of t...,24 hour room service|Airport transportation (s...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014042513...,NaN,12.978389,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Magic Inn - Marathahalli,Hotel,NaN,NaN,NaN,Magic Standard,NaN,4.5,0.032092
1,26755,"No:2, Ulsoor Road (Parallel to MG Road) , Ulso...","Ulsoor,Central Bangalore",Bangalore,"|Situated in the Silicon Valley of India, Shil...",Airport/Rlwy Stn Transfer|Business Centre|Curr...,2.0,//imghtlak.mmtcdn.com/hotels/20110121180320201...,NaN,12.975349,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Shilton Suites,Hotel,NaN,NaN,NaN,Deluxe Suite,NaN,4.5,0.047548
2,25175,"No 1, 1st Floor, Outer Ring Rd, Near to Relien...",Marathahalli,Bangalore,"Nestled in the city of Bangalore, Hotel Nandha...",Doctor on Call|Laundry Service|Restaurant|Room...,2.0,NaN,NaN,12.948576,...,https://hotelz.makemytrip.com/makemytrip/site/...,Nandhana Comforts Marathahalli,Hotel,NaN,NaN,NaN,Standard|Deluxe,Karnataka,4.5,0.053065
3,28828,"11/1, Hosur Main Road, In-Between Christ Unive...",Koramangala,Bangalore,Around 4 km from Lal Bagh and 9 km from the ra...,Airport Transfer|Doctor on Call|Laundry Servic...,2.0,NaN,NaN,12.935282,...,https://hotelz.makemytrip.com/makemytrip/site/...,The Libra Hotel,Hotel,NaN,NaN,NaN,Economy Ac|Executive Ac|Premium Ac,Karnataka,4.5,0.065624
4,20002,"44B, 1st Main Road, Opp. Mini Forest,3rd Phase...",JP Nagar,Bangalore,Situated at a 20 minute drive from Bangalore C...,Airport/Rlwy Stn Transfer|Coffee Shop|Conferen...,2.0,NaN,NaN,12.911775,...,http://hotelz.makemytrip.com/makemytrip/site/h...,The Grand Continent,Hotel,NaN,NaN,NaN,Deluxe King,Karnataka,4.5,0.087935


In [19]:
# Rename the column "hotel_facilities" to "facilities"
hotels_df = hotels_df.rename(columns={'hotel_facilities': 'facilities'})

In [20]:
hotels_df = hotels_df.rename(columns={'property_name': 'Hotel_name'})

In [21]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import MultiLabelBinarizer


# Define a function to calculate Jaccard similarity between a user's preferences and all hotels
def calculate_similarity(user_id):
    user_prefs = user_preferences_df[user_preferences_df['user_id'] == user_id].iloc[:, 3:].values
    hotel_features = hotels_df.iloc[:, 3:].values

    # Create a MultiLabelBinarizer object to one-hot encode the values in each column as binary features
    mlb = MultiLabelBinarizer()

    # Fit the MultiLabelBinarizer on all the values in the 'Facilities' column of the hotels dataframe
    mlb.fit(hotels_df['facilities'].str.split(', '))

    # Transform the 'Facilities' column of both user preferences and hotels dataframes into binary features
    user_facilities = mlb.transform(user_preferences_df[user_preferences_df['user_id'] == user_id]['facilities'].str.split(', '))
    hotel_facilities = mlb.transform(hotels_df['facilities'].str.split('|'))

    # Calculate Jaccard similarity between the user's preferences and all hotels
    similarities = pairwise_distances(user_facilities, hotel_facilities, metric='jaccard')

    return 1 - similarities[0]


In [22]:
hotels_df

,Unnamed: 0,address,area,city,hotel_description,facilities,hotel_star_rating,image_urls,locality,latitude,...,pageurl,Hotel_name,property_type,province,room_count,room_facilities,room_type,state,rating,Distance
0,9462,"39/5 - Doddanekkundi, Mahadevpura, Marathahall...","Marathahalli,Outer Ring Road,Accenture, RMZ Ec...",Bangalore,|Magic inn is situated at the primal part of t...,24 hour room service|Airport transportation (s...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014042513...,NaN,12.978389,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Magic Inn - Marathahalli,Hotel,NaN,NaN,NaN,Magic Standard,NaN,4.5,0.032092
1,26755,"No:2, Ulsoor Road (Parallel to MG Road) , Ulso...","Ulsoor,Central Bangalore",Bangalore,"|Situated in the Silicon Valley of India, Shil...",Airport/Rlwy Stn Transfer|Business Centre|Curr...,2.0,//imghtlak.mmtcdn.com/hotels/20110121180320201...,NaN,12.975349,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Shilton Suites,Hotel,NaN,NaN,NaN,Deluxe Suite,NaN,4.5,0.047548
2,25175,"No 1, 1st Floor, Outer Ring Rd, Near to Relien...",Marathahalli,Bangalore,"Nestled in the city of Bangalore, Hotel Nandha...",Doctor on Call|Laundry Service|Restaurant|Room...,2.0,NaN,NaN,12.948576,...,https://hotelz.makemytrip.com/makemytrip/site/...,Nandhana Comforts Marathahalli,Hotel,NaN,NaN,NaN,Standard|Deluxe,Karnataka,4.5,0.053065
3,28828,"11/1, Hosur Main Road, In-Between Christ Unive...",Koramangala,Bangalore,Around 4 km from Lal Bagh and 9 km from the ra...,Airport Transfer|Doctor on Call|Laundry Servic...,2.0,NaN,NaN,12.935282,...,https://hotelz.makemytrip.com/makemytrip/site/...,The Libra Hotel,Hotel,NaN,NaN,NaN,Economy Ac|Executive Ac|Premium Ac,Karnataka,4.5,0.065624
4,20002,"44B, 1st Main Road, Opp. Mini Forest,3rd Phase...",JP Nagar,Bangalore,Situated at a 20 minute drive from Bangalore C...,Airport/Rlwy Stn Transfer|Coffee Shop|Conferen...,2.0,NaN,NaN,12.911775,...,http://hotelz.makemytrip.com/makemytrip/site/h...,The Grand Continent,Hotel,NaN,NaN,NaN,Deluxe King,Karnataka,4.5,0.087935
5,25732,"No 354, Ashoka Road, Near St. Philomena Church...",Ashoka Road,Mysore,Hotel Sunshine is located at the most convenie...,24 hours power back-up|Doctor on Call|Elevator...,2.0,NaN,NaN,12.322325,...,https://hotelz.makemytrip.com/makemytrip/site/...,Hotel Sunshine,Hotel,NaN,NaN,NaN,Deluxe Non Ac|Deluxe Ac,Karnataka,4.5,1.197344
6,10532,"Opposite to KSRTC Bus Sta , Mysore",Mysore KSRTC Bus Station,Mysore,|Hotel Shashtri Paradise is a budget property ...,Doctor on Call|Laundry|Room Service|Hot/cold W...,2.0,//imghtlak.mmtcdn.com/images/hotels/2014121219...,NaN,12.311716,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Shastri Paradise,Hotel,NaN,NaN,NaN,Standard Room|Shastri Deluxe Double|Executive ...,NaN,4.5,1.202081
7,28761,"Shop No. 2911, Opp. KSRTC Bus Stand, BN Road M...",Mysore KSRTC Bus Station,Mysore,Hotel Shashtri Paradise is a budget property l...,24 hours power back-up|Air Conditioned|Doctor ...,2.0,NaN,NaN,12.311752,...,https://hotelz.makemytrip.com/makemytrip/site/...,Shastri Paradise,Hotel,NaN,NaN,NaN,Non Ac Standard|Deluxe Double Non Ac|Executive...,Karnataka,4.5,1.202062
8,10566,"2724/C, Bangalore-Nilgiri Road , Mysore","City Centre,Near Mysore Palace ,Central Mysore",Mysore,"|800 m from Mysore Palace, 2.3 km from Mysore ...",Board Room|Car Rental Facility|Coffee Shop|Con...,2.0,//imghtlak.mmtcdn.com/hotels/20070830162607268...,NaN,12.308411,...,http://hotelz.makemytrip.com/makemytrip/site/h...,Hotel Roopa,Hotel,NaN,NaN,NaN,Deluxe Non Ac|Executive Non Ac|Luxury Ac,NaN,4.5,1.203417
9,22916,"2722/1, L13/1 L/14, Sri Harsha Road , Mysore",Lasker Mohalla,Mysore,"|Ideally located on Sri Harsha Road, SK Elegan...",Airport Transfer|Doctor on Call|Guide Service|...,2.0,//imghtlak.mmtcdn.com/images/hotels/2012091018...,NaN,12.308620,...,http://hotelz.makemytrip.com/makemytrip/site/h...,S K Elegance,Hotel,NaN,NaN,NaN,Executive Non A/c|Executive Ac,NaN,4.5,1.204101


In [24]:
hotel_names = hotels_df['Hotel_name']

In [28]:
# Example usage: Recommend top 5 hotels for user 1
user_id = 1
similarities = calculate_similarity(user_id)
hotel_data = hotels_df[['Hotel_name', 'facilities','Hotel_name','property_type','rating', 'Distance']]
recommendations = pd.DataFrame({'Hotel_Name': hotel_names, 'Similarity': similarities})
recommendations = recommendations.merge(hotel_data, on='Hotel_name').sort_values(by='Similarity', ascending=False).head(5)


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['24-Hour Room Service', 'Air Conditioning', 'Airport Shuttle', 'Bar', 'Business Center', 'Free Parking', 'Gym', 'Housekeeping', 'Laundry Service', 'Pet-Friendly', 'Swimming Pool'] will be ignored
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) ['', ' 24-hr Room Service ', ' Air Conditioned ', ' Air Conditioning', ' Air Conditioning ', ' Air conditioning ', ' Attached Bathroom ', ' Bathroom Toiletries ', ' Cable T V', ' Cable T V ', ' Cable Tv', ' DVD Player on Request ', ' Daily Newspaper ', ' Desk ', ' Direct dial phone ', ' Equipped Kitchenette ', ' Fireplace ', ' Full-length mirror ', ' Geyser In Bathroom', ' Geyser In Bathroom ', ' Hot & Cold Running Water ', ' Hot/cold Water', ' Hot/cold Water ', ' Intercom ', ' Intercom Facility ', ' Iron/ironing board (on request)', ' Iron/ironing board (on request

ValueError: ignored